In [ ]:
# !pip install pydrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# auth.authenticate_user()
# gauth = GoogleAuth

# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# download = drive.CreateFile({'id': '1U3-XsgxUF_v_K4jXhwl6LJVty5eR9EUm'})
# download.GetContentFile('data.zip')
# ! unzip data.zip

In [ ]:
import numpy as np
import keras as ks
import cv2
import string
import os
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
import pickle
import keras
import nltk
nltk.download('punkt')

In [ ]:
cap_path="captions"

for i in os.listdir(cap_path):
      print(i)

In [ ]:
cap_path="captions"


train_ids=open(cap_path+'/Flickr_8k.trainImages.txt','r')
train_ids=train_ids.read()
train_ids=train_ids.strip().split('\n')

test_ids=open(cap_path+'/Flickr_8k.testImages.txt','r')
test_ids=test_ids.read()
test_ids=test_ids.strip().split('\n')

val_ids=open(cap_path+'/Flickr_8k.devImages.txt','r')
val_ids=val_ids.read()
val_ids=val_ids.strip().split('\n')


len(train_ids)+len(test_ids)+len(val_ids),len(train_ids),len(test_ids),len(val_ids)

In [ ]:
captions=open(cap_path+'/Flickr8k.lemma.token.txt','r')
captions=captions.read()
captions=captions.split('\n')
captions=[i.split('#') for i in captions]
captions=[(i[0],i[1].split('\t')[1]) for i in captions if len(i)>1]
capt=[]
i=0
while i<len(captions):
    li=[]
    j=1
    while(i+j<len(captions) and captions[i][0]==captions[i+j][0]):
        li.append(captions[i+j][1])
        j+=1
    capt.append((captions[i][0],li))
    i+=j
captions=capt
len(captions)


In [ ]:
train_data=[]
val_data=[]
test_data=[]
for i in range(len(captions)):
    if captions[i][0] in train_ids:
        train_data.append(captions[i])
    elif captions[i][0] in test_ids:
        test_data.append(captions[i])
    elif captions[i][0] in val_ids:
        val_data.append(captions[i])
    else:
        continue
len(train_data)+len(val_data)+len(test_data),len(train_data),len(val_data),len(test_data)

In [ ]:
num=['0','1','2','3','4','5','6','7','8','9']

def data_making(data):
    clean_data=[]
    clean_labels=[]
    for i in range(len(data)):
        
        for j in data[i][1]:
            j=j+" endseq"
            for k in num:
                if k in j:
                    j=j.replace(k," ")
            
            for k in string.punctuation:
                if k in j:
                    j=j.replace(k," ")
            
            k=nltk.tokenize.word_tokenize(j)
            for m in range(len(k)):
              k[m]=k[m].lower()
              
            l=[]
            for p in k:
                if len(p)<2:
                    continue
                l.append(p)
            d=[data[i][0],"startseq"]
            clean_data.append(d)
            clean_labels.append(l[0])
            for k in range(len(l)-1):
                d=clean_data[-1].copy()
                
                d.append(l[k])
                
                clean_data.append(d)
                clean_labels.append(l[k+1])
            
    return clean_data,clean_labels           
            

In [ ]:
train_data,train_labels=data_making(train_data)
val_data,val_labels=data_making(val_data)
test_data,test_labels=data_making(test_data)
len(train_data)+len(val_data)+len(test_data),len(train_data),len(val_data),len(test_data)


In [ ]:

# # https://drive.google.com/open?id=1N6efgxjRdQExrz7z84_iJiUGXIWad6FU
# download = drive.CreateFile({'id': '1N6efgxjRdQExrz7z84_iJiUGXIWad6FU'})
# download.GetContentFile('image_vectors.pickle')
# file=open("image_vectors.pickle","rb")
# image_vectors=pickle.load(file)
# image_vectors[train_data[0][0]]

In [ ]:
vocab=train_labels+['startseq']
vocab=list(set(vocab))
len(vocab),vocab[:1]

In [ ]:

def get_mapping(vocab):
    j=1
    mapping={}
    for i in vocab:
        mapping[i]=j
        j+=1
    print(j)
    return mapping

def get_demapping(vocab,mapping):
    demapping={}
    for i in vocab:
        demapping[mapping[i]]=i
    return demapping


In [ ]:
# mapping=get_mapping(vocab)
# demapping=get_demapping(vocab,mapping)
# file=open("mappings_more.pickle","wb")
# pickle.dump([mapping,demapping],file)
# file.close()
 
# # https://drive.google.com/open?id=1Aie1lKhWpkrj3nm3ATiRf9LJ_r7RJ0HR
# # less-> https://drive.google.com/open?id=173ZBh6CGgwHuUrvtqWjKPFCiwtvVOq_T
# # download = drive.CreateFile({'id': '173ZBh6CGgwHuUrvtqWjKPFCiwtvVOq_T'})
# # download.GetContentFile('mappings_less.pickle')
# # file=open("mappings_less.pickle","rb")
# # mapping,demapping=pickle.load(file)   
# # file.close()

In [ ]:
vosize=len(demapping)+1
demapping[vosize-1],mapping["be"],vosize,len(vocab),mapping["the"]

In [ ]:
# words to integers
for i in train_data:
    for j in range(1,len(i)):
        i[j]=mapping[i[j]]
        
train_labels_clean=[]
        
for i in range(len(train_labels)):
        train_labels_clean.append(mapping[train_labels[i]])


In [ ]:

dataset_images=[]
dataset_words=[]

for i in train_data:
    dataset_words.append(i[1:])
for i in train_data:
    dataset_images.append(image_vectors[i[0]])
 #padding flatten-images=>204800[20,20,512],sequences=>[35]
dataset_words[:5]

In [ ]:
from keras.preprocessing.sequence import pad_sequences
#dataset_images=pad_sequences(dataset_images,padding='post')
dataset_words=pad_sequences(dataset_words,padding='post',maxlen=38)

In [ ]:
from keras.layers import Input,Dense,SimpleRNN,Dropout,Embedding,GRU,LSTM,add,concatenate
from keras.models import Model

In [ ]:
encoder=Input(shape=(38,))
emb=Embedding(input_dim=vosize+2,output_dim=256,mask_zero=True)(encoder)
drop=Dropout(rate=0.25)(emb)
RNN_en=SimpleRNN(256,return_sequences=False)(drop)

image_encoder=Input(shape=(4096,),dtype="float32")
drop_i=Dropout(rate=0.25)(image_encoder)
dense_i=Dense(256,activation=ks.activations.relu)(drop_i)

added=concatenate([RNN_en,dense_i])


dense1=Dense(256,activation=ks.activations.relu)(added)

dense2=Dense(vosize,activation=ks.activations.softmax)(dense1)


model=Model(inputs=[encoder,image_encoder],outputs=dense2)
opti=ks.optimizers.adam()
model.compile(loss=ks.losses.sparse_categorical_crossentropy,optimizer=opti,metrics=['accuracy'])

In [ ]:
dataset_images=np.array(dataset_images)
dataset_words=np.array(dataset_words)
train_labels_clean=np.array(train_labels_clean)
train_labels_clean=train_labels_clean.reshape((train_labels_clean.shape[0],1))
dataset_images=dataset_images.reshape((dataset_images.shape[0],4096))
dataset_images.shape,dataset_words.shape,train_labels_clean.shape

In [ ]:
# # https://drive.google.com/open?id=1yLO5apOdVrlTBqYvw0lGZVozkHg1NFDQ
# download = drive.CreateFile({'id': '1yLO5apOdVrlTBqYvw0lGZVozkHg1NFDQ'})
# download.GetContentFile('mymodel_new.h5')
# model=keras.models.load_model('mymodel_new.h5')


# download = drive.CreateFile({'id': '1yLO5apOdVrlTBqYvw0lGZVozkHg1NFDQ'})
# download.GetContentFile('mymodel_less.h5')
# model=keras.models.load_model('mymodel_less.h5')



In [ ]:
model.fit(x=[dataset_words,dataset_images],y=train_labels_clean,epochs=5,verbose=1,batch_size=2048)#20 (min80% accuracy)

In [ ]:
  def captioner(start,get_vector):
    i=1
    text=""
    while(i<33):
#         print(start.shape,get_vector.shape)
        pred=model.predict([start,get_vector])
       
#         print(pred.shape)
        pred=np.argmax(pred[0])
        start[0][i]=pred
        
#         print(get_word(pred))
        if(demapping[pred]=="endseq"):
            break
        text=text+" "+demapping[pred]
        i+=1
    return text

In [ ]:
for i in range(0,len(dataset_images)-250,250):
  start=np.zeros((1,38))
  start[0][0]=mapping["startseq"]
  # start[0][1]=get_word_id("A")+1
  # start[0][1]=get_word_id("black")
  # start[0][1]=get_word_id("dog")
  a=captioner(start,dataset_images[i:i+1])
  print(a,"\n")

In [ ]:


# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)  

# model.save("mymodel_imgcapt_more.h5")
# # get the folder id where you want to save your file
# file = drive.CreateFile()
# file.SetContentFile("mymodel_imgcapt_more.h5")
# file.Upload()